In [120]:
import cv2
from ultralytics import YOLO
import cvzone
import math

from sort import *

In [121]:
# creating web cam object
# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture('./Videos/cars.mp4')
#cap.set(3,1280) # for width
#cap.set(4,720) #for height

In [122]:
model = YOLO('./Yolo-Weights/yolov8l.pt')

In [123]:
# classes that will be detected and displayed
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush" , "pen"
              ]

In [124]:
mask = cv2.imread('./mask.png')

In [125]:
# creating a instance for tracking
tracker = Sort(max_age=20,min_hits=3,iou_threshold=0.3)

# we will make a line and count after that 
limits = [400,297,673,297]
totalcounts=[]

In [126]:
while True:
    success, img = cap.read()
    #overlay mask on the image/webcam feed
    imageRegion = cv2.bitwise_and(img,mask)
    # results = model(img,stream=True)
    results = model(imageRegion,stream=True)
    
    detections = np.empty((0,5))
    
    for r in results:
        boxes = r.boxes
        for box in boxes:
            # xy1 are points and xy2 are height and width
            x1,y1,x2,y2 = box.xyxy[0]
            # converting them into integers instead for tensor 
            x1,y1,x2,y2 = int(x1),int(y1),int(x2),int(y2)
            # cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,255),3) #using cv2 reactangle
            w,h = x2-x1 , y2-y1
            bbox = int(x1),int(y1),int(w),int(h)
            
            # print(x1,y1,x2,y2)
            
            # cvzone.cornerRect(img,(x1,y1,w,h),l=9) #using cvzone rectange
            
            #confidence
            conf = math.ceil((box.conf[0]*100))/100 #for confidence values
            print(conf)
            #classnames
            cls = int(box.cls[0])
            #to display conf and object class on webcam
            currentclass = classNames[cls]
            
            if currentclass == "car" or currentclass == "truck" or currentclass == "bus" or currentclass == "motorbike" and conf>0.3 :
                # cvzone.putTextRect(img,f'{currentclass} {conf}',(max(0,x1),max(35,y1)) , scale=0.6,thickness=1,offset=3)
                # cvzone.cornerRect(img,(x1,y1,w,h),l=9 , rt=5) #using cvzone rectange
                currentArrays = np.array([x1,y1,x2,y2,conf])
                detections = np.vstack((detections,currentArrays))
            
    resultsTracker = tracker.update(detections)
    
    cv2.line(img,(limits[0],limits[1]),(limits[2],limits[3]),(0,0,255),5)
    
    for result in resultsTracker:
        x1,y1,x2,y2,id = result
        # converting them into integers instead for tensor 
        x1,y1,x2,y2 = int(x1),int(y1),int(x2),int(y2)
        print(result)
        w,h = x2-x1 , y2-y1
        cvzone.cornerRect(img,(x1,y1,w,h),l=9 , rt=2,colorR=(255,0,0))
        # cvzone.putTextRect(img,f' {int(id)}',(max(0,x1),max(35,y1)) , scale=2,thickness=3,offset=10)
        
        cx , cy = x1+w//2,y1+h//2
        cv2.circle(img,(cx,cy),5,(255,0,255),cv2.FILLED)
        
        if limits[0] <cx< limits[2] and limits[1]-20<cy<limits[1]+20:
            # totalcounts.append(id)
            # cvzone.putTextRect(img,f' {int(totalcounts)}',(max(0,x1),max(35,y1)) , scale=2,thickness=3,offset=10)
            if totalcounts.count(id)==0:
                totalcounts.append(id)
                cvzone.putTextRect(img,f' {len(totalcounts)}',(max(0,x1),max(35,y1)) , scale=2,thickness=3,offset=10)
                
        
    cvzone.putTextRect(img,f'Count : {len(totalcounts)}',(90,90)) 
    
    cv2.imshow("Image",img)
    # cv2.imshow("ImageRegion",imageRegion)
    # cv2.waitKey(1)
    # Wait for a key event, but only for a short duration (10 milliseconds)
    # If the key is Esc (27), then break out of the loop and close the webcam
    if cv2.waitKey(1) == 27:
        break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


0: 384x640 3 cars, 830.9ms
Speed: 14.6ms preprocess, 830.9ms inference, 24.6ms postprocess per image at shape (1, 3, 384, 640)



0.81
0.8
0.67
[        571         316         651         429         692]
[        378         348         495         457         691]
[        458         226         509         270         690]


0: 384x640 3 cars, 932.9ms
Speed: 8.0ms preprocess, 932.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)



0.88
0.83
0.82
[     568.92      322.12      649.08      435.88         692]
[     373.98      352.02      493.02      462.98         691]
[     456.14      227.88      506.86      271.12         690]


0: 384x640 4 cars, 740.5ms
Speed: 0.0ms preprocess, 740.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)



...

0.81
0.79
0.74
0.73
0.73
0.72
[     630.27      199.32      666.29       220.7         727]
[     516.15      200.35      559.32      232.86         724]
[      526.5      250.13      579.09      292.81         722]
[     310.72      346.75       407.2      419.67         714]
[     628.49      258.06      664.61      303.42         713]


0: 384x640 6 cars, 1 truck, 717.7ms
Speed: 3.5ms preprocess, 717.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)



0.84
0.81
0.76
0.75
0.66
0.58
0.46
[     628.64       200.1      667.87      225.74         727]
[     513.36      201.69      557.72      236.26         724]
[      518.1       259.2      573.59      304.67         722]
[     284.05      364.97      388.89      444.22         714]
[     622.38      269.09       662.7      317.62         713]


0: 384x640 6 cars, 1 truck, 709.4ms
Speed: 8.0ms preprocess, 709.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)


0.84
0.81
0.77
0.76
0.66
0.6
0.47
[     579.88      198.77      617.54      221.93         728]
[     626.53      203.13      666.33      231.64         727]
[     510.35      202.84      555.78      239.43         724]
[     508.62      269.55      566.95      317.57         722]
[     254.31      382.04      367.58         463         714]
[     615.65      281.96      660.52      334.25         713]
